# VideoRcvKafka
The producer (VideoSendKafka.py) is sending video frames on the TOPIC,consume and render
the frames as they arrive. The frames are encoded and broken up into chunks.

Demotrate that we can send encoded images 

In [ ]:
import kafka
import os
import getpass
import sys
import json
import base64
import kafka
import ssl
import time
import datetime
import matplotlib.pyplot as plt
import io
from PIL import Image
import logging
import numpy as np
import cv2
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import Button, HBox, VBox, Layout
from IPython.display import display, clear_output

if '../jupyter' not in sys.path:
    sys.path.insert(0, '../jupyter')
import credential

In [ ]:
## setup 

TOPIC = 'VideoFrame'

def decode_img(bin64):
    img = Image.open(io.BytesIO(base64.b64decode(bin64)))
    return img

## Get the credentials.

In [ ]:
creds = json.loads(credential.magsEventStream)

## Kafka arrive, decode and display
The producer (VideoSendKafka.py) is sending video frames on the TOPIC, here consume and write
the frames as they arrive. 

In [ ]:
img_obj = widgets.Output(layout={'border': '1px solid red','width':'50%','height':'300pt'})
dashboard_img = widgets.VBox([img_obj])
display(dashboard_img)


cons = kafka.KafkaConsumer(TOPIC, 
                              bootstrap_servers=creds["kafka_brokers_sasl"],
                              security_protocol="SASL_SSL",
                              sasl_mechanism="PLAIN",
                                   sasl_plain_username=creds["user"],
                                   sasl_plain_password=creds["api_key"],
                                   ssl_cafile=ssl.get_default_verify_paths().cafile, consumer_timeout_ms=30000)

start = time.time()
logging.getLogger("__name__").warning("start")
startup = True     # get to begining of frame at startup.
for msg in cons:
    chunk = json.loads(msg[6])
    if startup: 
        if chunk['chunk_idx'] != 0:
            continue
        startup = False
    if chunk['chunk_idx'] == 0:
        # start of a new frame.
        image_string = chunk['data']
        continue
    if chunk['chunk_idx'] == chunk['chunk_total']:
        # frame complete - convert to image and display
        image = decode_img(image_string)
        with img_obj:
            display(image)
            clear_output(wait=True)
        chunk_string = ""
        continue
    # building up frame
    image_string = "".join([image_string, chunk['data']])    
    
    
    print("message cnt:", len(chunks))
cons.close()
logging.getLogger("__name__").warning("done")
